Import needed set of packages

In [ ]:
import datetime
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from tools import rednt_tools as tools  # custom helper tools library

Read the data from the CSV files with timestamp as index column

In [ ]:
v_df = pd.read_csv(tools.VIBRATION_FILENAME, sep=tools.SEP, header=None,
                   names=tools.V_HEADER_NAMES, index_col=[0], parse_dates=[0],
                   date_parser=tools.date_parser)

t_df = pd.read_csv(tools.TEMPERATURE_FILENAME, sep=tools.SEP, header=None,
                   names=tools.T_HEADER_NAMES, index_col=[0], parse_dates=[0],
                   date_parser=tools.date_parser)

Make sure that both of the data are sorted

In [ ]:
v_df = v_df.sort_values(by='timestamp')
t_df = t_df.sort_values(by='timestamp')

Determine some timestamp characterstics - start time, end time and time horizon of signals

In [ ]:
timestamp_chars = {
    'vibration': {
        'start': v_df.index.to_series()[0],
        'end': v_df.index.to_series()[-1],
        'horizon': v_df.index.to_series()[-1] - v_df.index.to_series()[0]
    },
    'temperature': {
        'start': t_df.index.to_series()[0],
        'end': t_df.index.to_series()[-1],
        'horizon': t_df.index.to_series()[-1] - v_df.index.to_series()[0]
    }
}

print("Timestamp characteristics:")
print(timestamp_chars)

Calculate signal timestamp differences between samples and some statistics about it

In [ ]:
v_timestamp_diff = v_df.index.to_series().diff()
t_timestamp_diff = t_df.index.to_series().diff()

timestamp_diff = pd.concat([v_timestamp_diff, t_timestamp_diff])
timedelta_value_counts = timestamp_diff.value_counts()

print('Timestamp differences:')
print(timedelta_value_counts)

timedelta_statistics = {
    'mean': timestamp_diff.mean(),
    'median': timestamp_diff.median(),
    'mode': timestamp_diff.mode(),
    'max': timestamp_diff.max(),
    'min': timestamp_diff.min()
}

print('Timestamp differences statistics:')
print(timedelta_statistics)

Deduce sampling time with timedelta as the median from the diffs

In [ ]:
timedelta = timedelta_statistics['median']

print("Median Timedelta that we choose as sampling time: {}".format(timedelta))
sampling_time = timedelta

Upsample the missing data with the NaNs

In [ ]:
v_df = v_df.resample(sampling_time).mean()
t_df = t_df.resample(sampling_time).mean()

Merge dataframes based on the closest timestamp and limit to smallest length

In [ ]:
merge_tolerance = sampling_time
if len(v_df) > len(t_df):
    df = pd.merge_asof(t_df, v_df, on='timestamp', tolerance=merge_tolerance)
else:
    df = pd.merge_asof(v_df, t_df, on='timestamp', tolerance=merge_tolerance)

Plot the timeseries data

In [ ]:
tools.plot_timeseries_data(df)